**Проект: Анализ базы данных приложения для чтения книг по подписке**

**Цель проекта: Анализ базы данных с целью представления информации о книгах, издательствах, авторах, а также пользовательских обзорах книг для формулирования ценностных предложений для нового продукта.**

В ходе проекта были изучены таблицы базы. 

Также определено, сколько книг вышло после 1 января 2000 года; количество обзоров и среднюю оценку для каждой книги; издательство, которое выпустило наибольшее число книг толще 50 страниц; автор с самой высокой средней оценкой книг; среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

Изучаю таблицы базы. Вывожу первые строки таблиц.

In [3]:
print('Таблица "Books"')
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

Таблица "Books"


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
print('Таблица "authors"')
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

Таблица "authors"


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
print('Таблица "publishers"')
query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

Таблица "publishers"


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
print('Таблица "ratings"')
query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

Таблица "ratings"


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
print('Таблица "reviews"')
query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''
pd.io.sql.read_sql(query, con = engine)

Таблица "reviews"


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Определение параметров по заданию

Определяю количество книг, вышедших с 1 января 2000 года

In [8]:
query1 = ''' SELECT COUNT(*)
            FROM books
            WHERE publication_date > '2000.01.01'
        '''
pd.io.sql.read_sql(query1, con = engine)

,count
0,819


В базе данных есть информация о 819 книг, изданных после 01.01.2000

Определяю для каждой книги посчитайте количество обзоров и среднюю оценку, сортирую по среднему рейтингу

In [9]:
query2 = ''' WITH 
            c_r AS 
            (SELECT book_id,
            COUNT(review_id) AS count_review
            FROM reviews
            GROUP BY book_id),
            
            r AS
            (SELECT book_id,
            AVG(rating) AS mean_raiting
            FROM ratings
            GROUP BY book_id)
            
            SELECT   books.title
                    ,c_r.count_review
                    , r.mean_raiting
                FROM books 
            LEFT OUTER JOIN c_r ON books.book_id=c_r.book_id
            LEFT OUTER JOIN r ON books.book_id=r.book_id
            ORDER BY mean_raiting
            
            
        '''
pd.io.sql.read_sql(query2, con = engine)

,title,count_review,mean_raiting
0,Harvesting the Heart,2.0,1.50
1,Drowning Ruth,3.0,2.00
2,His Excellency: George Washington,2.0,2.00
3,Junky,2.0,2.00
4,The World Is Flat: A Brief History of the Twen...,3.0,2.25
...,...,...,...
995,How to Be a Domestic Goddess: Baking and the A...,1.0,5.00
996,March,2.0,5.00
997,Arrows of the Queen (Heralds of Valdemar #1),2.0,5.00
998,Moneyball: The Art of Winning an Unfair Game,3.0,5.00


In [10]:
# сортирую по количеству обзоров
query2 = ''' WITH 
            c_r AS 
            (SELECT book_id,
            COUNT(review_id) AS count_review
            FROM reviews
            GROUP BY book_id),
            
            r AS
            (SELECT book_id,
            AVG(rating) AS mean_raiting
            FROM ratings
            GROUP BY book_id)
            
            SELECT   books.title
                    ,c_r.count_review
                    , r.mean_raiting
                FROM books 
            LEFT OUTER JOIN c_r ON books.book_id=c_r.book_id
            LEFT OUTER JOIN r ON books.book_id=r.book_id
            ORDER BY count_review
            
            
        '''
pd.io.sql.read_sql(query2, con = engine)

,title,count_review,mean_raiting
0,The Road Not Taken and Other Poems,1.0,3.5
1,Debt of Honor (Jack Ryan #7),1.0,3.0
2,Stone of Farewell (Memory Sorrow and Thorn #2),1.0,5.0
3,Ranma ½ Vol. 1 (Ranma ½ (US 2nd) #1),1.0,4.0
4,Executive Orders (Jack Ryan #8),1.0,3.5
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,NaN,4.0
996,Leonardo's Notebooks,NaN,4.0
997,Essential Tales and Poems,NaN,4.0
998,The Natural Way to Draw,NaN,3.0


В базе данных есть 1 книга с наименьшим рейтингом 1,5 и 2 обзорами. Максимальныйсредний  рейтинг - 5,0.  Наибольшее число обзоров - 7, есть книги на которых обзоров нет. Среди таковых есть книги с высоким рейтингом. 

Определяю издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query3 = ''' 
            
            WITH m as
            (SELECT publishers.publisher,
            COUNT(book_id) as num_book
            FROM books
            LEFT OUTER JOIN publishers AS publishers ON publishers.publisher_id=books.publisher_id
            WHERE num_pages>50
            GROUP BY publishers.publisher)
            
            SELECT *
            FROM m
            where num_book = (Select max(num_book) from m)
            
            

            
        '''
pd.io.sql.read_sql(query3, con = engine)

,publisher,num_book
0,Penguin Books,42


Книг издательства Penguin Books больше всего в базе (исключая издательства выпускающе брошюры). В базе 42 книги этого издательства.

Определяю автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [15]:
query4 = ''' 
           WITH list as 
           (SELECT book_id, avg(R.rating) as avg_book_raiting
           FROM ratings AS R
           GROUP BY book_id
           HAVING COUNT(rating_id)>50)
           
           SELECT author, AVG(avg_book_raiting) as avg_raiting
           FROM list
           LEFT JOIN Books as b on list.book_id = b.book_id
           LEFT JOIN AUTHORS AS A ON a.author_id = b.author_id
           GROUP BY author
           ORDER BY avg_raiting DESC
           LIMIT 1
           
           
        '''
pd.io.sql.read_sql(query4, con = engine)

,author,avg_raiting
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самым высоким средним рейтингом книг, среди популярных книг (книги которых были оценены больше 50 раз) -  J.K. Rowling/Mary GrandPré. Средняя оценка книг этого автора - 4,28

Считаю среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query5 = '''
            SELECT AVG(d.num_review) AS avg_review
            FROM
            (SELECT rating_num.username, review_mean.num_review
            FROM
            
            (SELECT *
            FROM
            (SELECT username, COUNT(rating_id) as num_rating
            FROM ratings
            GROUP BY username) as list
            WHERE num_rating>50) AS rating_num
            
            LEFT OUTER JOIN
            
            (SELECT username, COUNT(review_id) as num_review
            FROM reviews
            GROUP BY username) AS review_mean
            
            ON rating_num.username=review_mean.username) as d

            
        '''
pd.io.sql.read_sql(query5, con = engine)

,avg_review
0,24.333333


# Вывод

В базе данных есть информация о 819 книгах, выпущенных после 01.01.2000 года. Самая низкая средняя оценка - 1,5. Самая высокая - 5,0. Есть книги на которые нет обзоров. Максимальное число обзоров у книги- 7. Больше всего книг у издательства Penguin Books - 42. J.K. Rowling/Mary GrandPr - автор с самой высокой средней оценкой книг (4,28). В приложении есть 6 пользователей, оставивие более 50 оценок. Среднее число обзоров от этих пользователей - 24,33.